
# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset is extracted from an Udacity's competition called 'Predicting customer satisfaction' It consists of customers data from Santander bank, containing all their relevant features (like the number of products, their name, interest rate of loans, etc). 

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
cluster_name = 'project-cluster'
try:
    compute_target = ComputeTarget(ws, cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
#import dataset
from azureml.data.dataset_factory import TabularDatasetFactory
Data_URL= "https://github.com/jfcarmonag/Capstone_AZMLND/raw/master/starter_file/BankChurners.csv"
ds=TabularDatasetFactory.from_delimited_files(path=Data_URL)
# choose a name for experiment
experiment_name = 'exp-capstone'
experiment=Experiment(ws, experiment_name)

In [4]:
import pandas as pd
from train import clean_data
from azureml.train.automl import AutoMLConfig

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
complete_data = pd.concat([x, y], axis=1)

complete_data

,Customer_Age,Gender_M,Gender_F,Dependent_count,Education_Level,Married,Unknown,Divorced,Single,Income_Category,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,45,1,0,3,1,1,0,0,0,3,...,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0
1,49,0,1,5,3,0,0,0,1,1,...,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0
2,51,1,0,3,3,1,0,0,0,4,...,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0
3,40,0,1,4,1,0,1,0,0,1,...,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0
4,40,1,0,3,0,1,0,0,0,3,...,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,50,1,0,2,3,0,0,0,1,2,...,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0
10123,41,1,0,2,0,0,0,1,0,2,...,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,1
10124,44,0,1,1,1,1,0,0,0,1,...,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,1
10125,30,1,0,2,3,0,1,0,0,2,...,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

- Since the dataset is unbalanced (HOW), F1-score is appropriate given that it takes into account both precision and recall.

In [17]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

from azureml.core import Workspace, Dataset
local_path = 'data/prepared.csv'
complete_data.to_csv(local_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/prepared.csv'))])

Uploading an estimated of 1 files
Uploading data/prepared.csv
Uploaded data/prepared.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [18]:
import pandas as pd
from train import clean_data
from azureml.train.automl import AutoMLConfig



# TODO: Put your automl settings here
automl_settings = {
       "n_cross_validations": 3,
       "primary_metric": 'accuracy',
       "enable_early_stopping": True,
       "experiment_timeout_hours": 1.0,
       "max_concurrent_iterations": 4,
       "max_cores_per_iteration": -1,
       #"verbosity": logging.INFO,
   }
# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                               compute_target = compute_target,
                               training_data = dataset,
                               label_column_name = "Attrition_Flag",
                               **automl_settings
                               )

In [19]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output="True")

Running on remote.
No run_configuration provided, running on project-cluster with default configuration
Running on remote compute: project-cluster
Parent Run ID: AutoML_04282637-4153-44f6-9f9a-0aa040ef4887

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+----

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, best_model = remote_run.get_output()

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = "model.pkl", model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service